In [5]:
import sys
sys.path.insert(1, 'C:\\dev\\pytrends')

from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=360)

In [3]:
date = '2015'
df = pytrends.top_charts(date, hl='en-US', tz=360, geo='AU')
if df is None:
    print('no data')
else:
    print(df.head())

             title exploreQuery
0          Netflix             
1  Rugby World Cup             
2       Lamar Odom             
3     Ronda Rousey             
4        Asian Cup             


In [66]:
import time

start_keywords = ['deep learning']

d = {}

def trend(keywords):
    pytrends.build_payload(kw_list=keywords)
    related = pytrends.related_queries()
    return related
  
def search(keywords, max_queries):
    visited = []
    queue = []
    links = []
    query_count = 0
    for keyword in keywords:
        queue.append(keyword)
    while len(queue) > 0 and query_count < max_queries:
        query_keywords = []
        # search for the next 5 keywords
        while len(queue) > 0 and len(query_keywords) < 5:
            keyword = queue.pop(0)
            if not keyword in visited:
                query_keywords.append(keyword)
        if len(query_keywords) > 0:
            # Ask google (uses batches of 5 to limit the number of requests)
            print('Query', query_keywords)
            results = trend(query_keywords)
            time.sleep(1)
            query_count += 1
            # 
            for keyword, dfs in results.items():
                visited.append(keyword)
                top_df = dfs['top']
                if top_df is None:
                    return links
                top = top_df.to_numpy()
                print('top', top)
                for top_keyword, value in list(top)[:5]:
                    if not top_keyword in visited:
                        print('Should visit', top_keyword)
                        queue.append(top_keyword)
                for top_keyword, value in list(top)[:5]:
                    links.append((keyword.lower(), keyword, top_keyword, value))
        print('queue', queue)
        print('query_count', query_count)
    return links

links = search(start_keywords, 6)
print('ok')

Query ['Australia']
top [['india australia' 100]
 ['australia time' 68]
 ['australia vs india' 68]
 ['sydney australia' 48]
 ['australia post' 40]
 ['melbourne australia' 38]
 ['time in australia' 36]
 ['australia cricket' 35]
 ['australia news' 32]
 ['ebay australia' 30]
 ['visa australia' 28]
 ['new zealand' 28]
 ['western australia' 27]
 ['australia day' 27]
 ['perth australia' 23]
 ['google australia' 22]
 ['india australia live' 21]
 ['virgin australia' 21]
 ['australia map' 21]
 ['coronavirus australia' 19]
 ['australia weather' 18]
 ['amazon australia' 17]
 ['bank australia' 17]
 ['pakistan vs australia' 16]
 ['india vs australia live' 15]]
Should visit india australia
Should visit australia time
Should visit australia vs india
Should visit sydney australia
Should visit australia post
queue ['india australia', 'australia time', 'australia vs india', 'sydney australia', 'australia post']
query_count 1
Query ['india australia', 'australia time', 'australia vs india', 'sydney austr

top [['what time in australia' 100]
 ['what time is in in australia' 93]
 ['what is time in australia' 90]
 ['time is it in australia' 70]
 ['what time is it' 67]
 ['time now in australia' 50]
 ['australia time now' 49]
 ['time in australia sydney' 41]
 ['sydney time' 41]
 ['time in sydney' 40]
 ['the time in australia' 39]
 ['current time in australia' 30]
 ['australia current time' 29]
 ['time in melbourne' 25]
 ['time in melbourne australia' 25]
 ['melbourne time' 25]
 ['australia time melbourne' 25]
 ['australia time right now' 25]
 ['time right now in australia' 24]
 ['what the time in australia' 22]
 ['what is the time in australia' 19]
 ['what time in australia now' 18]
 ['what is time now in australia' 16]
 ['time in perth australia' 14]
 ['what time in sydney australia' 14]]
Should visit what time in australia
Should visit what time is in in australia
Should visit what is time in australia
Should visit time is it in australia
Should visit what time is it
top [['time in austral

top [['time in australia' 100]
 ['what time australia' 36]
 ['time is australia' 35]
 ['what time in australia' 31]
 ['what time is australia' 30]
 ['what is time in australia' 27]
 ['what time is is in australia' 27]
 ['time now australia' 21]
 ['what time is it australia' 21]
 ['what time is it' 20]
 ['sydney' 19]
 ['australia time sydney' 19]
 ['sydney time' 19]
 ['sydney australia' 18]
 ['time now in australia' 14]
 ['australia melbourne time' 13]
 ['melbourne' 12]
 ['australia melbourne' 12]
 ['melbourne time' 12]
 ['time in sydney' 12]
 ['time in sydney australia' 12]
 ['the time in australia' 12]
 ['australia current time' 11]
 ['time of australia' 10]
 ['current time in australia' 8]]
Should visit what time is australia
top [['time in australia sydney' 100]
 ['time in australia' 98]
 ['time in sydney' 98]
 ['what time sydney australia' 33]
 ['what time in sydney' 31]
 ['what time in australia' 31]
 ['what time in australia sydney' 31]
 ['what time is sydney australia' 30]
 ['wh

In [64]:
print(links)

[('data science', 'Data Science', 'python', 100), ('data science', 'Data Science', 'data science python', 100), ('data science', 'Data Science', 'data science course', 94), ('data science', 'Data Science', 'data science jobs', 89), ('data science', 'Data Science', 'computer science', 89), ('python', 'python', 'python for', 100), ('python', 'python', 'python list', 83), ('python', 'python', 'python string', 76), ('python', 'python', 'python if', 43), ('python', 'python', 'install python', 43), ('data science python', 'data science python', 'python for data science', 100), ('data science python', 'data science python', 'python in data science', 72), ('data science python', 'data science python', 'data science with python', 41), ('data science python', 'data science python', 'python and data science', 37), ('data science python', 'data science python', 'machine learning', 27), ('data science course', 'data science course', 'course in data science', 100), ('data science course', 'data scie

In [67]:
import json

def node(x):
    return {
        'id': x,
        'group': 0
    }

def edge(x):
    return {
        'source': x[0],
        'target': x[2],
        'value': x[3]
    }

unique_nodes = set([x[0] for x in links] + [x[2] for x in links])

doc = { 
    'nodes': [node(x) for x in unique_nodes],
    'links': [edge(x) for x in links] 
}

# This is ready for visualization
print(json.dumps(doc))


{"nodes": [{"id": "score india australia", "group": 0}, {"id": "parcel tracking", "group": 0}, {"id": "india vs australia match", "group": 0}, {"id": "live india vs australia score", "group": 0}, {"id": "india live score", "group": 0}, {"id": "what time is in in australia", "group": 0}, {"id": "india vs australia cricket", "group": 0}, {"id": "australia vs india live score", "group": 0}, {"id": "match india vs australia", "group": 0}, {"id": "time in australia", "group": 0}, {"id": "time is it in australia", "group": 0}, {"id": "score india vs australia live", "group": 0}, {"id": "post tracking number", "group": 0}, {"id": "cricket live", "group": 0}, {"id": "what time is is in australia", "group": 0}, {"id": "what time is it australia", "group": 0}, {"id": "sydney australia time", "group": 0}, {"id": "post tracking", "group": 0}, {"id": "what time is it", "group": 0}, {"id": "match india australia", "group": 0}, {"id": "what time it is in australia", "group": 0}, {"id": "cricket", "gr